In [ ]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/minsir/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "minsirx.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()

import imaplib
import email
import pandas as pd
from datetime import datetime
import csv
from pandas import DataFrame

from email.header import decode_header
from bs4 import BeautifulSoup
import re
from apps.email_manager.models import EmailData, EmailAttachment
from django.core.files.base import ContentFile
import pytz

# what is the longest length of convo
# how to extract one conversations from the data

In [ ]:
class EmailNode:
    def __init__(self, previous_node, email_data, next_node):
        self.previous_node: EmailNode = previous_node
        self.email_data: EmailData = email_data
        self.next_node: EmailNode = next_node

class EmailConvo:
    def __init__(self, first_email):
        email_node = EmailNode(
            previous_node=None,
            email_data=first_email,
            next_node=None
        )
        self.head_email_node = email_node
        self.tail_email_node = email_node
    
    def append_email_node_to_convo(self, new_email):
        next_node = EmailNode(
            previous_node=self.tail_email_node,
            email_data=new_email,
            next_node=None
        )
        self.tail_email_node.next_node = next_node
        self.tail_email_node = next_node

    def prepend_email_node_to_convo(self, new_email):
        first_node = EmailNode(
            previous_node=None,
            email_data=new_email,
            next_node=self.head_email_node
        )
        self.head_email_node.previous_node = first_node
        self.head_email_node = first_node

    def remove_last_email(self):
        new_last_email = self.tail_email_node.previous_node
        if new_last_email is not None:
            new_last_email.next_node = None
            self.tail_email_node = new_last_email

    def remove_first_email(self):
        new_first_email = self.head_email_node.next_node
        if new_first_email is not None:
            new_first_email.previous_node = None
            self.head_email_node = new_first_email
    
    def remove_email_from_convo(self, index = None, e_id = None):
        if (index is not None and id is not None) or (index is None and id is None):
            return -1
        iter = 0
        def _remove_email_from_convo(current_node: EmailNode):
            nonlocal iter
            if (e_id is not None and current_node.email_data.id == e_id) or (index is not None and iter==index):
                previous_node: EmailNode = current_node.previous_node
                next_node: EmailNode = current_node.next_node
                if previous_node is not None and next_node is not None:
                    previous_node.next_node = next_node
                    next_node.previous_node = previous_node
                elif previous_node is not None and next_node is None:
                    previous_node.next_node = None
                elif previous_node is None and next_node is not None:
                    next_node.previous_node = None
                    self.head_email_node = next_node
                elif previous_node is None and next_node is None:
                    return 0
                return 1
            else:
                next_node = current_node.next_node
                if next_node is not None:
                    iter = iter + 1
                    return _remove_email_from_convo(next_node)
                else:
                    return -1
        return _remove_email_from_convo(self.head_email_node)
    
    def insert_email_into_convo(self, new_email: EmailData, index = None, e_id = None):
        if (index is not None and id is not None) or (index is None and id is None):
            return -1
        iter = 0
        def _insert_email_into_convo(current_node: EmailNode):
            nonlocal iter
            if (e_id is not None and current_node.email_data.id == e_id) or (index is not None and iter==index):
                new_email_node = EmailNode(
                    previous_node=None,
                    email_data=new_email,
                    next_node=None
                )
                previous_node: EmailNode = current_node.previous_node
                next_node: EmailNode = current_node.next_node
                if previous_node is not None:
                    previous_node.next_node = new_email_node
                    new_email_node.previous_node = previous_node

                    current_node.previous_node = new_email_node
                    new_email_node.next_node = current_node

                else:
                    current_node.previous_node = new_email_node
                    new_email_node.next_node = current_node
                    self.head_email_node = new_email_node
                return 1
            else:
                next_node = current_node.next_node
                if next_node is not None:
                    iter = iter + 1
                    return _insert_email_into_convo(next_node)
                else:
                    return -1
        return _insert_email_into_convo(self.head_email_node)
    
    def retrieve_email_from_convo(self, index = None, e_id = None, text = None):
        if (index is None and id is None and text is None):
            return -1
        iter = 0
        def _retrieve_email_from_convo(current_node: EmailNode):
            nonlocal iter
            if (
                (e_id is not None and current_node.email_data.id == e_id) 
                or (index is not None and iter==index) 
                or (current_node.email_data.body is not None and text in current_node.email_data.body)
            ):
                return current_node.email_data
            else:
                next_node = current_node.next_node
                if next_node is not None:
                    iter = iter + 1
                    return _retrieve_email_from_convo(next_node)
                else:
                    return None
        return _retrieve_email_from_convo(self.head_email_node)
            

        


In [ ]:
# if it is of the form re, but before the re, there is one without the re, then split
class ConversationReader:
    def __init__(self):
        self.conversations = {}
        self.latest_subjects = {}

    def extract_conversation(self):
        emails = EmailData.objects.order_by('date').all()[0:500]
        for an_email in emails:
            if an_email.subject and an_email.date:
                is_reply = False
                is_fwdd = False
                original_subject = an_email.subject.strip()
                if "RE: " in original_subject:
                    is_reply = True
                if "FW: " in original_subject:
                    is_fwdd = True
                rmvd_subject = original_subject.replace("RE: ", "")
                rmvd_subject = original_subject.replace("FW: ", "")
                if rmvd_subject in self.latest_subjects:
                    if not is_reply and not is_fwdd:
                        latest_date = an_email.date
                        self.latest_subjects[rmvd_subject] = latest_date
                    else:
                        latest_date = self.latest_subjects[rmvd_subject]
                else:
                    latest_date = an_email.date
                    self.latest_subjects[rmvd_subject] = latest_date
                formatted_date = latest_date.strftime('%B %d, %Y, %-I:%M %p')
                key_str = f"{formatted_date}@{rmvd_subject}"
                if key_str in self.conversations:
                    convo_emails: EmailConvo = self.conversations[key_str]
                    convo_emails.append_email_node_to_convo(an_email)
                else:
                    self.conversations[key_str] = EmailConvo(first_email=an_email)
                


In [ ]:
convo_reader = ConversationReader()
convo_reader.extract_conversation()